In [1]:
# Step 1: Mount Google Drive and unzip the dataset
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
drive_path = '/content/drive/MyDrive/DLHW6'
zip_file_path = f'{drive_path}/archive.zip'
unzip_folder_path = '/content/dataset'

# Unzip the dataset
print("Unzipping dataset, this may take a while...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder_path)
print("Unzipping complete.")

# Define paths for REAL and FAKE test folders
test_real_dir = os.path.join(unzip_folder_path, 'test/REAL')
test_fake_dir = os.path.join(unzip_folder_path, 'test/FAKE')

Mounted at /content/drive
Unzipping dataset, this may take a while...
Unzipping complete.


In [2]:
!pip install transformers

In [3]:
# Step 2: Load the SDXL model and set up the GPU device
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the model and processor from Hugging Face, sending the model to the GPU
model_name = "Organika/sdxl-detector"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

# Function to classify an image using the SDXL model
def classify_image(image_path):
    # Open and preprocess the image using the AutoImageProcessor
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)  # Move inputs to GPU

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        _, predicted = torch.max(outputs.logits, 1)

    return predicted.item()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [4]:
# Step 3: Calculate detection accuracy for "REAL" images
real_correct = 0
real_total = 0

print("Evaluating REAL images...")
for image_name in os.listdir(test_real_dir):
    image_path = os.path.join(test_real_dir, image_name)
    prediction = classify_image(image_path)

    # "1" corresponds to "human" in the model's output classes
    if prediction == 1:
        real_correct += 1
    real_total += 1

real_accuracy = (real_correct / real_total) * 100
print(f"REAL Detection Accuracy: {real_accuracy:.2f}%")


Evaluating REAL images...
REAL Detection Accuracy: 87.14%


In [5]:
# Step 4: Calculate detection accuracy for "FAKE" images

fake_correct = 0
fake_total = 0

print("Evaluating FAKE images...")
for image_name in os.listdir(test_fake_dir):
    image_path = os.path.join(test_fake_dir, image_name)
    prediction = classify_image(image_path)

    # "0" corresponds to "artificial" in the model's output classes
    if prediction == 0:
        fake_correct += 1
    fake_total += 1

fake_accuracy = (fake_correct / fake_total) * 100
print(f"FAKE Detection Accuracy: {fake_accuracy:.2f}%")


Evaluating FAKE images...
FAKE Detection Accuracy: 27.68%
